
# ACS 1: Armageddon - The Hazard of Small Asteroids

[Gareth Collins](http://www.imperial.ac.uk/people/g.collins), [Thomas Davison](http://www.imperial.ac.uk/people/thomas.davison), [James Percival](http://www.imperial.ac.uk/people/j.percival) and [Rhodri Nelson](http://www.imperial.ac.uk/people/rhodri.nelson)

## Synopsis:

Asteroids entering Earth’s atmosphere are subject to extreme drag forces that decelerate, heat and disrupt the space rocks. The fate of an asteroid is a complex function of its initial mass, speed, trajectory angle and internal strength. 

[Asteroids](https://en.wikipedia.org/wiki/Asteroid) 10-100 m in diameter can penetrate deep into Earth’s atmosphere and disrupt catastrophically, generating an atmospheric disturbance ([airburst](https://en.wikipedia.org/wiki/Air_burst)) that can cause [damage on the ground](https://www.youtube.com/watch?v=tq02C_3FvFo). Such an event occurred over the city of [Chelyabinsk](https://en.wikipedia.org/wiki/Chelyabinsk_meteor) in Russia, in 2013, releasing energy equivalent to about 520 [kilotons of TNT](https://en.wikipedia.org/wiki/TNT_equivalent) (1 kt TNT is equivalent to $4.184 \times 10^{12}$ J), and injuring thousands of people ([Popova et al., 2013](http://doi.org/10.1126/science.1242642); [Brown et al., 2013](http://doi.org/10.1038/nature12741)). An even larger event occurred over [Tunguska](https://en.wikipedia.org/wiki/Tunguska_event), a relatively unpopulated area in Siberia, in 1908. 

<img src="images/chelyabinsk.png" width="640">

The purpose of this exercise is to (a) develop a fast numerical simulator to predict the fate of asteroids entering Earth’s atmosphere, and (b) use this simulator to develop a hazard mapper for an impact over the UK.


## Problem definition

### Equations

The dynamics of an asteroid in Earth’s atmosphere prior to break-up is governed by a coupled set of ordinary differential equations:


<table width="600">
    <tbody>
        <tr style="background-color: #FFFFFF;">
            <td><img src="images/variables.png" width="220"> </td>
            <td>
\begin{aligned} 
\frac{dv}{dt} & = \frac{-C_D\rho_a A v^2}{2 m} + g \sin \theta \\
\frac{dm}{dt} & = \frac{-C_H\rho_a A v^3}{2 Q} \\
\frac{d\theta}{dt} & = \frac{g\cos\theta}{v} - \frac{C_L\rho_a A v}{2 m} - \frac{v\cos\theta}{R_P + z} \\
\frac{dz}{dt} & = -v\sin\theta \\
\frac{dx}{dt} & = \frac{v\cos\theta}{1 + z/R_P}
\end{aligned}
            </td>
        </tr>
    </tbody>
</table>

In these equations, $v$, $m$, and $A$ are the asteroid speed (along trajectory), mass and cross-sectional area, respectively. We will assume an initially **spherical asteroid** to convert from initial radius to mass (and cross-sectional area). $\theta$ is the meteoroid trajectory angle to the horizontal (in radians), $x$ is the downrange distance of the meteoroid from its entry position, $z$ is the altitude and $t$ is time; $C_D$ is the drag coefficient, $g$ is the surface gravity, $\rho_a$ is the atmospheric density (a function of altitude), $C_H$ is an ablation efficiency coefficient, $Q$ is the specific heat of ablation; $C_L$ is a lift coefficient; and $R_P$ is the planetary radius. All terms use MKS units.


A commonly used criterion for the break-up of an asteroid in the atmosphere is when the ram pressure of the air interacting with the asteroid $\rho_a v^2$ first exceeds the strength of the asteroid $Y$.

$$\rho_a v^2 = Y$$

Should break-up occur, the asteroid deforms and spreads laterally as it continues its passage through the atmosphere. Several models for the spreading rate have been proposed. In the simplest model, the fragmented asteroid’s spreading rate is related to its along trajectory speed [(Hills and Goda, 1993)](http://doi.org/10.1086/116499):

$$ \frac{dr}{dt} = \left[\frac{7}{2}\alpha\frac{\rho_a}{\rho_m}\right]^{1/2} v$$

Where $r$ is the asteroid radius, $\rho_m$ is the asteroid density (assumed constant) and $\alpha$ is a spreading coefficient, often taken to be 0.3. It is conventional to define the cross-sectional area of the expanding cloud of fragments as $A = \pi r^2$ (i.e., assuming a circular cross-section), for use in the above equations. So, the originally spherical asteroid spreads laterally, flattening into a "pancake". Fragmentation and spreading **ceases** when the ram pressure drops back below the strength of the meteoroid $\rho_a v^2 < Y$.


The figure below shows a typical solution to the above set of equations for an impact airburst scenario. The kinetic energy loss per unit height, which is a good approximation for the energy transferred to the atmosphere as heat (and is often referred to as the energy deposited per km altitude), is shown as a function of altitude. 

<img src="images/airburst.png" width="640">

In this scenario the asteroid experiences breakup at approximately 48-km altitude (denoted by the star), before spreading and decelerating rapidly until an altitude of approximately 15 km at which point the **peak energy loss per unit height is maximum**. This point is often considered to be the **burst altitude**. The **total kinetic energy lost** by the asteroid at this point is a good estimate of the **airburst energy** (i.e., the total energy deposited into the atmosphere) for hazard analysis.

Impact scenarios with a burst altitude above the surface can be considered to be **airburst events**. Many airburst events do not form a sizable crater on the ground, because even if a substantial fraction of the original asteroid survives to the ground it will be decelerated to a very small fraction of its initial speed.

Impact scenarios with a burst altitude below the surface (i.e., peak energy deposition per unit height is not reached before the asteroid strikes the ground) will form a sizable impact crater on the ground and can be considered as a **cratering event** (although a sizable proportion of the original kinetic energy of the asteroid may be transferred to the air).

There are some more complex scenarios where a **low altitude airburst combined with a substantial crater-forming event** is likely. This regime is not well understood and for simplicity we will **not consider it here**.


The rapid deposition of energy in the atmosphere is analogous to an explosion and so the environmental consequences of the airburst can be estimated using empirical data from atmospheric explosion experiments [(Glasstone and Dolan, 1977)](https://www.dtra.mil/Portals/61/Documents/NTPR/4-Rad_Exp_Rpts/36_The_Effects_of_Nuclear_Weapons.pdf).

The main cause of damage close to the impact site is a strong (pressure) blastwave in the air, known as the **airblast**. Empirical data suggest that the pressure in this wave $p$ (in Pa) (above ambient, also known as overpressure), as a function of explosion energy $E_k$ (in kilotons of TNT equivalent), burst altitude $z_b$ (in m) and horizontal range $r$ (in m), is given by:

\begin{equation*}
p(r) = 3.14 \times 10^{11} \left(\frac{r^2 + z_b^2}{E_k^{2/3}}\right)^{-1.3} + 1.8 \times 10^{7} \left(\frac{r^2 + z_b^2}{E_k^{2/3}}\right)^{-0.565}
\end{equation*}

For airbursts, we will take the total kinetic energy lost by the asteroid at the burst altitude as the burst energy $E_k$. For cratering events, we will define $E_k$ as the **larger** of the total kinetic energy lost by the asteroid at the burst altitude or the residual kinetic energy of the asteroid when it hits the ground.

The following threshold pressures can then be used to define different degrees of damage.

|  Damage Level |  Description    | Pressure (kPa) |
|:-------------:|:---------------:|:--------------:|
|  1  |  ~10% glass windows shatter    |     1.0      |
|  2  | ~90% glass windows shatter     |     3.5      |
|  3  | Wood frame buildings collapse  |     27      |
|  4  | Multistory brick buildings collapse  |     43      |

<p>
<div align="center">Table 1: Pressure thresholds (in kPa) for airblast damage</div>

According to the equations that we will use in this work, an asteoroid of approximately 7-m radius is required to generate overpressures on the ground exceeding 1 kPa, and an asteoroid of approximately 35-m radius is required to generate overpressures on the ground exceeding 43 kPa.

An example of an airburst event energetic enough to generate overpressures exceeding 43 kPa on the ground is shown below. The circles of different colours denote the different blast zones. The black line is the part of the great circle between the location where the meteoroid entered the atmosphere (initial altitude of 100 km) and the surface zero location (closest point on surface to the burst point).

<img src="images/blast_damage.png" width="640">


## Challenge

Your task is to develop a Python program with two main features: 

1. The ability to solve the system of differential equations describing meteoroid entry and compute the burst altitude, burst energy and horizontal path length from the entry point.
2. The ability to take these outputs and a location in the UK and determine the predicted extent of airblast damage on the ground and the postcodes and population affected.

In the following, we describe the functionality that we would like you to incorporate into these two features. 

### Airburst solver

#### Core functionality

Your impact solver tool must take the following inputs

* Meteoroid radius (m)
* Meteoroid speed (m/s)
* Meteoroid density (kg/m$^3$)
* Meteoroid strength (Pa)
* Meteoroid trajectory angle (degrees)
* Solution **output** timestep (s)

and return a Pandas dataframe with the following columns:

* Time (s), altitude (m), horizontal position (m), speed (m/s), mass (kg), radius (m), kinetic energy loss per unit height (kt/km)

and a dictionary of outcomes:

* Whether the scenario is an airburst **or** a cratering event
* The peak kinetic energy loss per unit height in kt per km (value at ground if cratering event)
* The burst altitude in m (0 if cratering event)
* The total kinetic energy loss at burst in kt (airburst)
* The larger of the total kinetic energy loss or the residual kinetic energy in kt at impact with ground (cratering)
* The horizontal path length across Earth's surface in m from the entry point to the burst point (impact point for a cratering event)

You should also develop a simple interface for a User to interact with your tool (e.g., jupyter notebook, command-line arguments or example python scripts) and documented functionality to produce simple graphical output of the evolution of the asteroid in the atmosphere (e.g., plots of mass, speed, energy, etc., as a function of altitude or time).

Three python function interfaces and a class have been specified for you in `armageddon/solver.py` to use for the airburst solver. Further details are given in the [AirburstSolver.ipynb](./AirburstSolver.ipynb) notebook.


#### Extension Functionality

Additional credit will be given if your airburst solver tool demonstrates the following extended capabilities:

* Ability to use a tabulated atmospheric density profile instead of an exponential atmosphere (a tabulated atmospheric profile for Earth is provided in [here](./resources/AltitudeDensityTable.csv)).

* Ability to determine asteroid parameters (e.g., strength and radius) that best fit an observed energy deposition curve. As an example, you can use the energy deposition curve inferred for the Chelyabinsk event (available [here](./resources/ChelyabinskEnergyAltitude.csv)). Note that in this data file energy is given in units of [kilotons of TNT](https://en.wikipedia.org/wiki/TNT_equivalent), where 1 kt TNT is equivalent to $4.184 \times 10^{12}$ J. Note also that the density 3300 kg/m$^3$, impact angle (18.3 degrees to horizontal) and entry velocity (19.2 km/s) are well known from observations ([Popova et al., 2013](http://doi.org/10.1126/science.1242642)). 


### Airblast damage mapper

#### Core functionality

Your airblast damage mapper tool must take the following external inputs:

* Entry latitude (degrees as a decimal)
* Entry longitude (degrees as a decimal)
* Entry bearing (degrees from north, as a decimal)
* Entry altitude (m)

Plus the outputs from the airburst solver:

* Burst energy (kt TNT)
* Burst altitude (m)
* Horizontal path length (m)

And return the following information, in a series of functions:

* The surface zero location of the airburst in latitude and longitude
* The airblast damage radii for four different damage thresholds
* A list of lists that contains the postcodes (or post-code sectors) in England and Wales inside the radius of each airblast damage level (see Table 1). 
* A corresponding list of lists that contains the population of each postcode (or post-code sector) in England and Wales inside the radius of each airblast damage level. 

To achieve this functionality, you have been given two separate csv files. One file contains a list of **postcodes** in the UK together with the latitude and longitude of the centroid of the postcode. The other files contains population data for each **postcode sector** in England and Wales. These two datasets can be used to estimate the population in each postcode in England and Wales (assuming uniform population distribution within each postcode sector). Note that some data cleaning may be necessary to combine these datasets.

Further details are given in the [DamageMapper.ipynb](./DamageMapper.ipynb) notebook.

#### Extended functionality

Additional credit will be given if your damage mapper function demonstrates the following extended capabilities:

* The ability to present the software output on a map. The graphics should be designed to be appropriate for use in emergency response and evacuation planning.
* The ability to perform a simple uncertainty analysis that takes as input a small uncertainty on each input parameter and calculates a risk for each affected UK postcode (sector).

For this second extension exercise, a separate function should be written that takes an additional set of inputs, describing the standard deviation of each input parameter, as well as the nominal input parameters. The uncertainty in each input parameter can be assumed to follow a gaussian distribution centered on the nominal values. The standard deviations for the parameters can be taken as:

* Entry latitude 0.025$^\circ$
* Entry longitude: 0.025$^\circ$
* Entry bearing: 0.5$^\circ$
* Meteoroid radius: 1 m
* Meteoroid speed: 1000 m/s
* Meteoroid density: 500 kg/m$^3$
* Meteoroid strength: 50\%
* Meteoroid trajectory angle: 1$^\circ$

For the second extension task, risk will be defined as the probability that the postcode sector (or postcode) is within a specified damage zone times the affected population. This function should therefore take as an input the overpressure used in the risk calculation and a flag to indicate whether risk should be calculated at the postcode or postcode sector level. For scoring, we will use damage level 3 (wooden buildings collapse) and postcode sectors.

Your risk calculator should sample the model parameter space $n$ times, where $n$ is an input parameter, but the sampling method is up to you. The probability that a postcode (or sector) is within a specified damage level is defined as the number of times the postcode (sector) is within the specified damage level divided by $n$. 

Your function should output a Pandas dataframe with two columns: postcode (unit or sector) and risk.

## Assessment

The **deadline** for software submission is **Friday 25th November, 12:00 pm UTC**.

### Software (70 marks)

Your software will be assessed primarily on functionality (**30/70 marks**), performance (**20/70 marks**) and sustainability (**20/70 marks**).

Functionality (**30 marks**): Your software will be scored based on its ability to perform a number of automated tests. These are:

1. A comparison between your software output and an analytical solution (**3 marks**). The analytical solution is derived with the simplifying assumptions of exponential atmosphere ($\rho_a = \rho_0 e^{-z/H}$), no gravitational acceleration ($g=0$), a flat planet ($R_p = \infty$), no lift ($C_L = 0$), no mass change owing to ablation ($\frac{dm}{dt}=0$) and no fragmentation ($\sigma_0 = \infty$, $\frac{dr}{dt}=0$). Note that you should derive this analytical solution and incorporate this into your test framework. An initial altitude of $z_0 = 100,000$ m will be used for this test.

2. Correct solution for several sets of asteroid parameters (**7 marks**). Quality of the solution will be based on the error in burst altitude and peak energy deposition rate, as well as solution variables as a function of time. An initial altitude of $z_0 = 100,000$ m will be used for each of these tests.  The following scenarios will be tested:
    * Several impacts on Earth with different impactor parameters (from within a realistic distribution) and assuming an exponential atmosphere.
    * The same impact on Earth as one of the above scenarios, using the tabulated terrestrial atmosphere provided.


3. Correct calculation of the surface zero location and airblast damage radii for specified meteoroid parameters (**3 marks**)

4. Correct identification of postcodes inside each damage level for a specified impact scenario (meteoroid parameters and entry location and bearing), as well as the total affected population for each level (**7 marks**).

5. Correct identification of high risk postcode sectors for a specified impact scenario (meteoroid parameters and entry location and bearing) and specified uncertainties on each input parameter (**10 marks**).
   
Performance (**20 marks**): The time of completion of each of the tests in parts 2, 4 and 5 above will be used to measure the performance of your tool, both to calculate a single scenario and to calculate and synthesise the risk map.

Indicative scores of Functionality and Performance will be computed for tests 1, 2, 3 and 4 at two or three points during the week of the project. Test 5 will not be scored until after project submission. Note that the marks for Functionality and Performance will be based on these scores (i.e., higher score implies higher mark), but not necessarily in a linear mapping.

Sustainability (**20 marks**): As with all software projects, you should employ all the elements of best practice in software development that you have learned so far. A GitHub repository will be created for your project to host your software. The quality and sustainability of your software and its documentation will be assessed based on your final repository and how it evolves during the week. Specific attention will be given to the following elements:

1. Installation and usage instructions
2. Documentation (in HTML / PDF format). Note that a template SPHINX documentation framework has been provided for you as a starting point for your documentation.
3. Coding style
4. Quality and coverage of automatic testing framework
5. General repository usage
6. Licensing

Please refer to the ACDS handbook for more information about the assessment of software quality.

### Presentation (20 marks)

Your project will also be assessed on the basis of a 15-minute video presentation that you must save or upload your group's private Teams channel before the deadline of **Friday 25th November, 4:00 pm UTC**.

You can record the presentation in any software that you like, but we recommend recording in MS Teams as this allows for automatic uploading to MS Teams.

You presentation should provide the following information:

1. A brief description of your airburst solver solution algorithm, including ODE solving routine.
2. A quantification of the accuracy of your numerical solution for two cases, with and without fragmentation, for User-specified input parameters. It is up to you to design an appropriate demonstration of accuracy, but this could take the form of a plot of error vs. timestep size or error vs. solver tolerance, depending on your solution algorithm. 
3. A demonstration of how to use your software to determine the impactor parameters (asteroid size & strength) for the Chelyabinsk airburst, by fitting your model solutions of kinetic energy loss per unit height vs. altitude to the inferred energy deposition curve.
4. A brief description of your algorithm for finding postcodes within each damage zone
5. A brief description of your algorithm for accounting for the effect of uncertainty in the input parameters (risk calculation).
6. A demonstration of your software for a specified scenario that will be provided on Friday.


### Teamwork (peer assessment; 10 marks)

After the presentations, you will complete a self-evaluation of your group's performance. This will inform the teamwork component of your mark. Please refer to the ACDS guidelines for more information about the assessment of teamwork.

## Technical requirements

* You should use the assigned GitHub repository exclusively for your project
* Your software must be written to work in Python 3.8
* You are free to import anything from the standard python libraries as well as numpy, matplotlib, pandas, dask, scipy, mpltools and sympy (see the `requirements.txt` for the full list of preapproved packages in the environment).
* You have been given some geographic mapping examples using folium, but can request to use an alternative mapping package if you've used it previously. Requests should be submitted by 5pm GMT on Wednesday.
* For solving the ODEs you **must write your own ODE solution algorithm**. You can make use of any of your own implementations of algorithms that you have learned in Computational Maths, however you are not permitted to use the in-built ODE solvers in scipy (e.g., `odeint`, `solve_ivp`).
* You are not allowed to import other python packages without authorization (if in doubt, please query with the Module Coordinator)
* You can assume that Users of your software will have pytest installed, so this does not need to be part of your repository
* You should use GitHub Actions for any automated testing that you implement
* You do not need to make a Graphical User Interface for your software: the program can be run in the command line or in a Python 3.8 environment